## 实验：Affine 与 Softmax 层反向传播

### 实验概要

二层神经网络利用计算图的思想可以按照如下简单表示

<img src="./img/NN_01.jpg" width="60%">

上图中的黑色框均表示计算层，Affine 表示加权和层，ReLU 表示 ReLU 激活函数层，SoftmaxWithLoss 表示 Softmax 激活函数和 Loss 损失函数的组合层。

### 实验目标

在本实验中，我们将介绍如何在 Affine/Softmax 层的实现反向传播。

### 1. Affine 层实现

求加权和的过程就是 `X*W+B`。（X 表示数据矩阵，W 表示权重矩阵，B 表示偏置矩阵）

首先我们可以得到如下 Affine 层的具体计算图

<img src="./img/Affine_01.jpg" width="60%">

上图中每个确定的数据以及它的形状都已标识出来。(数据矩阵 **X** 形状假设为 $N*2$，权重矩阵 **W** 假设为 $2*3$，其中的 **L** 表示神经网络的最后输出）

灰色方框表示暂时还不确定的三个数据 `[方框1, 方框2, 方框3]`。下面我们来进行推理。

首先，根据如下数学式，已知 **X** 和 $\frac{\partial L}{\partial X}$ 形状一致，**W** 和 $\frac{\partial L}{\partial W}$ 形状一致，**B** 和 $\frac{\partial L}{\partial B}$ 形状一致。

$$x=(x_0,x_1,...,x_n)$$

$$\frac{\partial{L}}{\partial{X}}=(\frac{\partial{L}}{\partial{X_0}},\frac{\partial{L}}{\partial{X_1}},...,\frac{\partial{L}}{\partial{X_n}})$$

所以 **方框 1** 的形状是 $N*2$ ，**方框 2** 的形状是 $2*3$，**方框 3** 的形状是 $1*3$。

其次，如果上述 $X*W$ 仅是普通两个数字的乘法，那么 **方框 1** 就等于上游传来的导数 $\frac{\partial L}{\partial Y}$ 乘以 **W** (也就是乘以 $\frac{\partial X*W}{\partial X}$ )。虽然 $X*W$ 表示的是两个矩阵的乘法，不能直接如上计算。但是矩阵的乘法中包含了普通数字的乘法，所以可知 $\frac{\partial X*W}{\partial X}$ 必定与 **W** 有关。然后，我们再依据 **方框 1** 的形状和矩阵乘法中对应维度必须一致的要求，可以推得 **方框 1** 就是 $\frac{\partial L}{\partial Y}$ 乘以 **W** 的转置。同理可得，**方框 2** 就是 **X** 的转置乘以 $\frac{\partial L}{\partial Y}$ 。

最后，我们来考虑 **方框 3**。一般来说，矩阵的加法要求两个矩阵形状一致，当两个矩阵形状一致时，反向传播只需将上游传来的导数原封不动地传给下游。但是利用 Python 的广播功能，正向传播时 $X*W+B$ 是可以被正确计算的（ 也就是 $X*W（N*3)$ 当中的每一行都加了一次 $B(1*3)$ ）。对于此时的反向传播，$\frac{\partial L}{\partial Y(N*3)}$，也需要汇总为 $\frac{\partial L}{\partial B(N*3)}$，也就是 $\frac{\partial L}{\partial B}$ （**方框 3**）等于 $\frac{\partial L}{\partial Y}$ 垂直方向上的和。

### 2. 定义 Affine 类

In [1]:
# Affine层（计算加权和）
class Affine:
    # 加权和层初始化时要求入口参数为该层的权重W和偏置b
    def __init__(self,W,b):   
        self.W = W
        self.b = b
        # 用于计算反向传播时W的梯度
        self.x = None         
        # 用于存储反向传播时计算出的W的梯度
        self.dW = None        
        # 用于存储反向传播时计算出的b的梯度
        self.db = None        

    # 前向函数就是通过x,W和b计算出加权和，再输出
    def forward(self,x):       
        self.x = x
        out = np.dot(x,self.W) + self.b
        return out
     
     # 反向函数就是将上游传来的导数dout乘以权重矩阵的转置WT后输出
     #（反向函数的输出永远是上游传来的导数乘以该层正向输出对正向输入的偏导）
    def backward(self,dout): 
        dx = np.dot(dout,self.W.T)
        
        # 顺便计算出上游导数关于该层权重和偏置的导数，
        # 实际上就是求出了损失函数关于该层权重和偏置的梯度
        self.dW = np.dot(self.x.T,dout)  
        self.db = np.sum(dout,axis=0)
        return dx

### 3. Relu 层实现

ReLU 层的计算图就可以如下图表示：

<img src="./img/relu_01.jpg" width="60%">

### 4. 定义 Relu 类

In [2]:
# ReLU层
class ReLU:
    def __init__(self):
        # 初始化实例变量mask（mask用来区分输入数组中x<=0的情况）
        self.mask = None         

    # 前向函数，将输入数组x中小于等于0的部分置为0后输出
    # 前向函数forward的入口参数为调用该函数的对象self，和一个输入numpy数组(代表加权和数组)
    def forward(self,x):         
        # 用(x<=0)为实例变量mask赋值，(x<=0)表示一个boolean型的数组
        # （x中值<=0的位置为True，值>0的位置为False）
        self.mask = (x<=0)       
        # 定义输出信号out为一个与x相同的numpy数组
        out = x.copy()           
        # 将out数组中mask为True的地方设为0	
        out[self.mask] = 0       
        return out
    
    # 反向函数，将输入数组dout中正向传播时小于等于0的部分置0后输出
    # 后向函数backward的入口参数为调用该函数的对象self，和一个上游传来的导数dout
    def backward(self,dout):     
        # 将dout数组中mask为True的地方设为0
        dout[self.mask] = 0      
        dx = dout
        return dx

### 5. SoftmaxWithLoss 层的实现

SoftmaxWithLoss 层简易版计算图如下：

<img src="./img/softmax_01.jpg" width="60%">

SoftmaxWithLoss 层首先经过 Softmax 激活函数计算层将加权和形式解 **a** 转换为概率形式解 **y**，再经过 CrossEntropyError 损失函数计算层根据推理结果 **y** 和正确解标签 **t** 计算得到损失函数值 **L**，并输出。反向传播时，整个 SoftmaxWithLoss 层输入为 1（也就是 $\frac{\partial L}{\partial L}$），输出为推理结果与正确解的差分 $y-t$。

### 6. 定义 SoftmaxWithLoss 层

In [3]:
# SoftmaxWithLoss层（正规化和计算损失函数）
class SoftmaxWithLoss:
    def __init__(self):
        # 存储损失函数值
        self.loss = None 
        # 存储softmax的输出
        self.y = None    
        # 存储正确解标签（one-hot表示）
        self.t = None    

    # 前向函数，输出损失函数值
    def forward(self,x,t):    
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self.t)
        return self.loss

    # 反向函数，输出推理结果与正确解标签的差分
    def backward(self,dout=1):             
        # 用batch_size记录当前批数据的大小（一次处理的图像数目）
        batch_size = self.t.shape[0]        
        # 除以batch_size后得到的是单个图像的误差	
        dx = (self.y-self.t)/batch_size     
        return dx

### 实验小结

在本实验中，你实现了在 Affine/Softmax 层的实现反向传播。